In [21]:
push!(LOAD_PATH, "$(homedir())/Documents/RBC_Capstone/financial-opt-tools/src")

using JuPOT

# Generate synthetic data sets for Demonstration

############
# Asset #1 #
############
n = 10 # No. Of Assets
ret_1 = rand(n)
cov_1 = let
    S = randn(n, n)
    S'S + eye(n)
end
names_1 = [randstring(3) for i in 1:n] # List of asset names

# Assets data structure containing, names, expected returns, covariance
assets_1 = AssetsCollection(names_1, ret_1, cov_1) 

############
# Asset #2 #
############
n = 20 # No. Of Assets
ret_2 = rand(n)
cov_2 = let
    S = randn(n, n)
    S'S + eye(n)
end
target_return = 0.2
names_2 = [randstring(3) for i in 1:n] # List of asset names
# Assets data structure containing, names, expected returns, covariance
assets_2 = AssetsCollection(names_2, ret_2, cov_2) 




QQp    0.08806865261158969 
ZMB    0.6949216413385797 
UTZ    0.3145332076459437 
8CM    0.5642730051646005 
8xg    0.2884974955107298 
zOr    0.8998229318702884 
dLj    0.5671327597262696 
XOP    0.3695627194119302 
QgP    0.3675334876122882 
vPf    0.08253969957329166 
sU4    0.14715595205758292 
Eah    0.9000584570258121 
R2T    0.059162719401035124 
IzZ    0.415373210010761 
lCZ    0.2638403970867478 
oky    0.09278341751245422 
tc0    0.009032579565787957 
XTc    0.5493240824498342 
gBy    0.15654886985633643 
rAU    0.026900688370821824 


In [20]:
######################################################
################### SIMPLE MVO #######################
######################################################

# Example 1: Basic 

# refer to model definition for keyword arguments, etc.
mvo_1 = SimpleMVO(assets_1, target_return; short_sale=false)
sym_1 = Dict(:place => 0.1)
optimize(mvo_1, sym_1)

10-element Array{Float64,1}:
 0.0122635 
 0.0818057 
 1.95719e-9
 0.300784  
 0.0797592 
 0.161249  
 0.127173  
 0.0833155 
 0.0190067 
 0.134643  

In [6]:
##################################
# Example 2: Adding a Constraint #
##################################

# Adding a simple weight constraint
constraints_1 = Dict(:constraint1 => :(min_thresh .<= w .<= max_thresh))

syms_1 = Dict(:max_thresh => 0.5, :min_thresh => 0.1/n)

# refer to model definition for keyword arguments, etc.
mvo_1 = SimpleMVO(assets_1, target_return, constraints_1; short_sale=false)
optimize(mvo_1, syms_1)

LoadError: LoadError: in @addNLConstraint (0.01 .<= w .<= 0.5): only ranged rows of the form lb <= expr <= ub are supported.
while loading In[6], in expression starting on line 10

In [11]:
####################################################### 
# Example 3: Changing a Constraint's parameter values #
#######################################################

# Changing values of an entered constraint
syms_1["max_thresh"] = 0.9
syms_1["min_thresh"] = 0.2/n

# refer to model definition for keyword arguments, etc.
mvo_1 = SimpleMVO(assets_1, target_return, constraints_1; short_sale=false)
optimize(mvo_1, syms_1)

LoadError: LoadError: in @addNLConstraint (0.01 .<= w .<= 0.5): only ranged rows of the form lb <= expr <= ub are supported.
while loading In[11], in expression starting on line 8

In [10]:
####################################
# Example 4: Deleting a Constraint #
####################################

# Removing a previously defined constraint
delete!(constraints_1, "constraint1")
delete!(syms, "max_thresh")
delete!(syms, "min_thresh")

# refer to model definition for keyword arguments, etc.
mvo_1 = SimpleMVO(assets_1, target_return, constraints_1; short_sale=false)
optimize(mvo_1, syms_1)

LoadError: LoadError: in @addNLConstraint (0.01 .<= w .<= 0.5): only ranged rows of the form lb <= expr <= ub are supported.
while loading In[10], in expression starting on line 13

In [18]:
##########################################
# Example 5: Adding multiple Constraints #
##########################################

# Adding a multiple weight constraints
constraints_1 = Dict(:constraint1 => :(w[1:5] <= assets_1_5_max), :constraint2 => :(w[5:n] <= assets_5_n_max))

syms_1 = Dict(:assets_1_5_max => 0.3, :assets_5_n_max => 0.05)


# refer to model definition for keyword arguments, etc.
mvo_1 = SimpleMVO(assets_1, target_return, constraints_1; short_sale=false)
optimize(mvo_1, syms_1)

LoadError: LoadError: UndefVarError: n not defined
while loading In[18], in expression starting on line 13

In [16]:
#####################################
# Example 6: Using Different Assets #
#####################################

# Using the same previously defined constraints we can run the model on a different set of assets effortlessly
mvo_2 = SimpleMVO(assets_2, target_return, constraints_1; short_sale=false)
optimize(mvo_2, syms_1)

LoadError: LoadError: in @addNLConstraint (0.01 .<= w .<= 0.5): only ranged rows of the form lb <= expr <= ub are supported.
while loading In[16], in expression starting on line 7

In [20]:
##################################################
# Example 6: Using Different Sets of Constraints #
##################################################

# At this point we have defined a set of constraints called constraints
# JuPOT allows the cool feature of having multiple sets be seamlessly 

constraints_2 = Dict(:constraint => :(w .<= max_weight))
syms_2 = Dict(:max_weight => 0.5)

# Using Constraints set # 2 on asset set  # 2
mvo_2 = SimpleMVO(assets_2, target_return, constraints_2; short_sale=false)
optimize(mvo_2, syms_2)

# Forgot what constraints and symbols were defined for initial constraints? No Problem!
constraints_1 # Prints the constraints
sym_1 # prints the symboles

# Using Constraints set # 1 on asset set  # 2
mvo_2 = SimpleMVO(assets_2, target_return, constraints_1; short_sale=false)
optimize(mvo_2, syms_1)



LoadError: LoadError: in @addNLConstraint (w .<= 0.5): expected comparison operator (<=, >=, or ==).
while loading In[20], in expression starting on line 13

In [21]:
######################################################
################### Robust MVO #######################
######################################################

# refer to model definition for keyword arguments, etc
# If no uncertainty matrix is entered the model defaults
# to the ellipse whose axes are proportional to the 
# individual variances of each asset
rmvo_1 = RobustMVO(assets_1, target_return; short_sale=true)
optimize(rmvo, syms_1)

JuPOT.RobustMVO{Float64,ASCIIString}(:Min,[:(w[1:10])],:(dot(w,10x10 Array{Float64,2}:
 12.3644     3.52387  -0.163724   1.09899  …   0.19095    2.64892   -0.115527
  3.52387   10.157    -4.26591    1.96996      2.86068    3.1455    -2.50218 
 -0.163724  -4.26591  10.469      1.50287     -5.74508   -0.695578   5.94238 
  1.09899    1.96996   1.50287   10.2345      -1.38396    1.09894    1.58229 
 -2.58091   -6.92069   0.49264   -2.51137     -3.48893   -2.07226    1.93537 
  0.77658    1.35112   0.265068   1.45923  …   1.65674    4.08686    3.444   
  3.13629    4.36912   1.52137    2.67053      3.16866   -1.21913    0.777374
  0.19095    2.86068  -5.74508   -1.38396     11.5766     0.49449   -0.801254
  2.64892    3.1455   -0.695578   1.09894      0.49449   12.0134     3.49904 
 -0.115527  -2.50218   5.94238    1.58229     -0.801254   3.49904   13.8858   * w)),[:(dot([0.10716789982015662,0.3881088656876732,0.4710476165186779,0.6274702440071767,0.3236928628704714,0.673411583834904,0.759

5-element Array{Float64,1}:
 0.350672 
 0.148301 
 0.218071 
 0.204115 
 0.0788405